In [ ]:
import numpy as np
import pandas as pd

: 

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

: 

In [ ]:
movies = movies.merge(credits,on='title') #merging credit dataframe to movies dataframe 

: 

In [ ]:
movies.info()

: 

# DATA PROCESSING

In [ ]:
#Lets remove the column that are not required and only keep the useful column
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

: 

In [ ]:
movies.head()

: 

In [ ]:
movies.isnull().sum() #checking the null value

: 

In [ ]:
#There are threee movie in overview which have null value.So lets remove them
movies.dropna(inplace=True)

: 

In [ ]:
#checking the duplicate data in movies
movies.duplicated().sum()

: 

In [ ]:
#No column are duplicated.
movies.iloc[0].genres #taking the genres column from the 0th row

: 

In [ ]:
#The above format is in wierd format so we need to change the above format to the format of ['Action','Adventure','Fantasy','Science Fiction']

: 

In [ ]:
import ast
def convert(obj):  #Here obj is string
    L=[]
    for i in ast.literal_eval(obj): #ast.literal_eval(obj.This operation makes obj as a list and we can procees with the operation
        L.append(i['name'])
    return L 

: 

In [ ]:
movies['genres'] = movies['genres'].apply(convert) #movies ko genres column ma convert function apply gareko

: 

In [ ]:
movies.head(1)

: 

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

: 

In [ ]:
movies['cast'][0]

: 

In [ ]:
#malai mathi cast column ko bata suru ko 3rd dictionary matra chaieko ra harek dictionary ko name matra chaieko x
import ast
def convert3(obj):  #Here obj is string
    L=[]
    counter = 0
    for i in ast.literal_eval(obj): #ast.literal_eval(obj.This operation makes obj as a list and we can procees with the operation
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L 

: 

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

: 

In [ ]:
movies.head()

: 

In [ ]:
movies['crew'][0]

: 

In [ ]:
#from crew among many dictionary we only require the dictionary that has job as director
import ast
def fetch_director(obj):  #Here obj is string
    L=[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break  #movie ma euta matra director hunca so director vetepaxi loop bata niskine
    return L 

: 

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

: 

In [ ]:
movies.head(1)

: 

In [ ]:
#Here the colummn of overview is of string.So lets convert this overview to list so that we can concatenate it with other column
movies['overview'] = movies['overview'].apply(lambda x : x.split())

: 

In [ ]:
movies.head(1)

: 

In [ ]:
#Now lets apply transformation to the 4 columns to remove the space.
movies['genres'] = movies['genres'].apply(lambda x:[i .replace (" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i .replace (" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i .replace (" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i .replace (" ","") for i in x])

: 

In [ ]:
movies.head(1)

: 

In [ ]:
#Now lets concatenate the 5 column so that our dataframe only contains 3 columns i.e. movie_id,title,tags
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

: 

In [ ]:
movies.head(1)

: 

In [ ]:
#so we need only three column.So lets make a new dataframe that takes 3 column from movies
new_df = movies[['movie_id' , 'title' , 'tags']]

: 

In [ ]:
new_df

: 

In [ ]:
#now lets convert the list to the string
new_df['tags'] = new_df['tags'].apply( lambda x:" ".join(x))

: 

In [ ]:
new_df.head(1)

: 

In [ ]:
import nltk

: 

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

: 

In [ ]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

: 

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

: 

In [ ]:
new_df['tags'][0] #tags column ko 0th row

: 

In [ ]:
#Let convert every word in tags to lower case
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower() )

: 

In [ ]:
new_df.head(1)

: 

#  TEXT VECTORIZATION

In [ ]:
#ww are calculating the similarities between the texts of tags of different movies and find the similarities between movies.
#For this we convert the tags into vectors with the coordinate (total_similar_words,5000)
#since there are 5000 tag for 5000 movies so there are 5000 vectors for tag of movies
#We are using the concept called bag of words to convert text into vectors.According to this concept,for every tags of 5000 movies,we combine all words of the tags and ,
#and then we choose 5000 most common word from those collection of words of tags.And then after this i choose the tag of each movie and compare with these 5000 similar words and ,
#find how many words has been occured in tags of the movies from those collection of similar words in the form of vector.For eg in first movie name avatar there are 3 similar word from,
#5000 words so here one vector is (5000,3) and another movie is Spectre and there are 4 similar word in tags from the collection of similar words so the vector for second movi is made of (5000,4) and so on for'
#5000 movies 5000 vector are made.For n vector,the vector that has most nearest distance to that vector will be most similar movie.And we find cosine distance except eucledian distance to find distance between the vector.


: 

In [ ]:
#We will be removing the stop words like from,are,is ,an
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

: 

In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()

: 

In [ ]:
vectors #Here each item in array enclosed under [..] is one vector and there are total 4806 vector inside vectors

: 

In [ ]:
#here most element are zero because in 5000 word there are only 2 3 words that are similar in the tags.
#Now lets have a look to those 5000 similar words
feature_name = cv.get_feature_names_out()

: 

In [ ]:
for i in feature_name:
    print(i)

: 

In [ ]:
#Now since we have 5000 vectors for all movies.So we will be calculating cosine distance between each movies instead of eucladian distance.So more the cosine distance between two vectors,more less similarities.
#suruma tags haru maddhe 5000 similar word ko collection banaiyo.Ani tyo 5000 similar word haru lai feri individual movie tags sanga compare garera kati ota similar word x vanera vector ko form ma nikaliyo.Ra movie vector bich cosine distance thaapayera 2 movie bich similarities thapaiyo
#so for finding similarities using cosine distance lets import cosine_similarities function
from sklearn.metrics.pairwise import cosine_similarity

: 

In [ ]:
similarity=cosine_similarity(vectors) #vector have collection of 4806 vectors.So by using this function cosine_similarity(vectors) will be calculating the similarities between each vector with other vector and storing it in the variable similarity


: 

In [ ]:
similarity[0] #This will find the cosine similarities of each movie in a dataset with a movie in the given index

: 

In [ ]:
#Lets make a function in such way that If somebody give me the name of the movie then this function will be giving me the name of five similar movies
    #step to recommend the movies:
    #1)At first i will be finding the index of the movies.For eg the index of avatar movie is 0
    #2)Then i will be sorting the consine distance of the vector of movie with other movie such that the smallest distance will be in top and largest in last
    #3)The top 5 similar movies will be fetched and shown
def recommend(movie): 
    movie_index = new_df[new_df['title'] == movie].index[0] #Ask in chatgpt like this "What does this command do in jupyter movie_index = new_df[new_df['title'] == movie].index[0]"
    distances = similarity[movie_index] #This will return the array same as in cell no 102.So now lets sort this array but there is one problem
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:9]
    for i in movies_list:
        print(new_df.iloc[i[0]].title) 
        #Yo sabbai ekchoti vdeo ma hernu yo function ko bare matra

: 

In [ ]:
recommend('The Avengers')

: 

In [ ]:
#To send the name of the movies to the website lets use the pickle library
import pickle

: 

In [ ]:
pickle.dump(new_df,open('movie.pkl','wb')) #this will create the file named as movie.pk1 outside where it contains the dataframe new_df
#search in chatgpt "What does this command means in jupyter pickle.dump(new_df,open('movie.pk1','wb'))"

: 

In [ ]:
import os #To know the location of the notebook

notebook_path = os.getcwd()
print(notebook_path)

: 

In [ ]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb')) #converting the dataframe to dictionary and storing in movie_dict.pk1 in writing mode

: 

In [ ]:
#dataframe lai dictinary ma convert garexi data haru yesari store hunxa 
'''
DATAFRAME:
 'title': ['Avatar', 'Bob', 'Charlie'],
    'Age': [25, 30, 35],
    'City': ['New York', 'San Francisco', 'Los Angeles']

DICTIONARY:
{'title': {0: 'Avatar', 1: 'Bob', 2: 'Charlie'},
 'Age': {0: 25, 1: 30, 2: 35},
 'City': {0: 'New York', 1: 'San Francisco', 2: 'Los Angeles'}}
 
 ->For more information search in chatgpt as "How dataframe will be stored in form of dictionary in python"
 '''

: 

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb')) #similarity ko content lai similarity.pkl ma halera write mode ma kholeko

: 

: 

: 

: 

: 